In [36]:
import pandas as pd 
import json
from tqdm import tqdm
from datetime import datetime
import numpy as np


#### Citire fisiere Json

In [37]:
file_content = open('buildingsstate_data.json', encoding = 'utf-8')
date_scoli = json.loads(file_content.read())

In [38]:
test2 = open('school_details.json', encoding= 'utf8')
nume_scoli = json.loads(test2.read())

#### Selectie date din interiorul campurilor fisierelor Json

In [ ]:
util = []
for i in tqdm(range(len(date_scoli))):
    try:
        building_utilities = date_scoli[i]['buildingsstate']['schoolBuildingUtilities']
        if building_utilities  not in util:    
            util.append(date_scoli[i]['buildingsstate']['schoolBuildingUtilities'])
    except KeyError:
        print(f'KeyError at index {i} : buildingsstate not found')
        continue


In [ ]:
non_empty_util = [lst for lst in util if lst]
detalii_util = pd.DataFrame(util[0] for util in non_empty_util)
detalii_util['dateFrom'] = detalii_util['dateFrom'].apply(lambda x: datetime.utcfromtimestamp(x/1000))
detalii_util

In [ ]:
detalii_scoli = pd.DataFrame([element['details'] for element in nume_scoli])
detalii_scoli.drop(columns=['idSchoolYear', 'schoolNumbers'], inplace= True)
detalii_scoli

In [42]:
detalii_util.set_index('idSchool', drop= True, inplace= True)
detalii_scoli.set_index('idSchool',drop = True, inplace= True)

#### Centalizare date utilitati + date scoli

In [43]:
detalii_complete = pd.merge(detalii_scoli, detalii_util, how = 'left', left_index= True, right_index= True)

#### Citire fisier date BAC 2023 sesiunea 1

In [44]:
var = pd.read_excel(r'2023.07.19_bac_date-deschise_2023-ses1.xlsx')

#### Verificare Coduri SIRUES comune(coduri unice de identificare a unitatilor de invatamant din Romania)

In [ ]:
lista_coduri = []
for i in detalii_complete['siruesCode']:
    if i in var['Unitate (SIRUES)']:
        lista_coduri.append(i)
# var['Unitate (SIRUES)'] = var['Unitate (SIRUES)'].fillna(0).astype(int)
lista_coduri

#### Prelucrare coloane coduri SIURES(eliminare randuri din setul de date ce nu contin coduri SIURES)

In [81]:
detalii_complete['siruesCode'] = pd.to_numeric(detalii_complete['siruesCode'].fillna(0), errors = 'coerce')

detalii_complete['siruesCode'] = detalii_complete['siruesCode'].fillna(0).astype(int)

detalii_complete = detalii_complete[detalii_complete['siruesCode'] != 0]

In [65]:
var['Unitate (SIRUES)'] = pd.to_numeric(var['Unitate (SIRUES)'].fillna(0), errors = 'coerce')

var['Unitate (SIRUES)'] = var['Unitate (SIRUES)'].fillna(0).astype(int)

In [84]:
result = pd.merge(var, detalii_complete , how = 'left', left_on = 'Unitate (SIRUES)', right_on= 'siruesCode')

In [101]:
result.to_excel('Data Agregate(Octav).xlsx')

#### Verificare coduri unice elev duplicate(de investigat)

In [ ]:
coduri_candidat_duplicate = result['Cod unic candidat'].value_counts()[result['Cod unic candidat'].value_counts() > 5].index

result[result['Cod unic candidat'].isin(coduri_candidat_duplicate)]